In [1]:
# import libraries
library(arules)
library(TSP)
library(data.table)
library(ggplot2)
library(Matrix)
library(tcltk)
library(dplyr)
library(devtools)
library(purrr)
library(tidyr)
library(viridis)
library(networkD3)
# library(htmlwidgets)
library(magrittr)

Warning message:
"package 'arules' was built under R version 4.3.3"
Loading required package: Matrix

Warning message:
"package 'Matrix' was built under R version 4.3.3"

Attaching package: 'arules'


The following objects are masked from 'package:base':

    abbreviate, write


Warning message:
"package 'TSP' was built under R version 4.3.3"
Warning message:
"package 'ggplot2' was built under R version 4.3.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:arules':

    intersect, recode, setdiff, setequal, union


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'devtools' was built under R version 4.3.3"
Loading required package: usethis

Warning message:
"package 'usethis' was built under R version 4.3.3"

Attaching packa

In [2]:
transactions <- read.transactions("transactions.csv",
                                   rm.duplicates = FALSE, 
                                   format = "basket",  ##if you use "single" also use cols=c(1,2)
                                   sep=",",  ## csv file
                                   cols=1) ## The dataset HAS row numbers

In [3]:
# inspect data
inspect(transactions)

     items          transactionID
[1]  {bags,                      
      bodywash,                  
      drinks,                    
      electronics,               
      paper,                     
      pens}           shampoo    
[2]  {}               supplements
[3]  {electronics,               
      paper}          pens       
[4]  {bags,                      
      bodywash,                  
      electronics,               
      paper,                     
      shampoo,                   
      supplements}    cards      
[5]  {bags,                      
      bodywash,                  
      cards,                     
      drinks,                    
      electronics,               
      paper,                     
      shampoo}        pens       
[6]  {bags,                      
      bodywash,                  
      cards,                     
      paper,                     
      supplements}    shampoo    
[7]  {}               shampoo    
[8]  {pens,   

In [8]:
# create rules
rules = arules::apriori(transactions, parameter = list(support=0.2, confidence=.5, minlen=2))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen maxlen target  ext
        0.5    0.1    1 none FALSE            TRUE       5     0.2      2     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 6 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[9 item(s), 30 transaction(s)] done [0.00s].
sorting and recoding items ... [9 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3 4 done [0.00s].
writing ... [77 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


In [9]:
# inspect the rules
options(width=100)
inspect(rules)

     lhs                               rhs           support   confidence coverage  lift     count
[1]  {pens}                         => {drinks}      0.2000000 0.6666667  0.3000000 1.666667 6    
[2]  {drinks}                       => {pens}        0.2000000 0.5000000  0.4000000 1.666667 6    
[3]  {pens}                         => {electronics} 0.2000000 0.6666667  0.3000000 1.428571 6    
[4]  {pens}                         => {paper}       0.2000000 0.6666667  0.3000000 1.333333 6    
[5]  {cards}                        => {bags}        0.2333333 0.7777778  0.3000000 1.794872 7    
[6]  {bags}                         => {cards}       0.2333333 0.5384615  0.4333333 1.794872 7    
[7]  {cards}                        => {electronics} 0.2000000 0.6666667  0.3000000 1.428571 6    
[8]  {cards}                        => {paper}       0.2333333 0.7777778  0.3000000 1.555556 7    
[9]  {bodywash}                     => {shampoo}     0.2000000 0.6000000  0.3333333 1.500000 6    
[10] {sham

In [18]:
# inspect the rules with a dataframe
rules_df <- as(rules, "data.frame")

In [19]:
print(rules_df)

                                    rules   support confidence  coverage     lift count
1                      {pens} => {drinks} 0.2000000  0.6666667 0.3000000 1.666667     6
2                      {drinks} => {pens} 0.2000000  0.5000000 0.4000000 1.666667     6
3                 {pens} => {electronics} 0.2000000  0.6666667 0.3000000 1.428571     6
4                       {pens} => {paper} 0.2000000  0.6666667 0.3000000 1.333333     6
5                       {cards} => {bags} 0.2333333  0.7777778 0.3000000 1.794872     7
6                       {bags} => {cards} 0.2333333  0.5384615 0.4333333 1.794872     7
7                {cards} => {electronics} 0.2000000  0.6666667 0.3000000 1.428571     6
8                      {cards} => {paper} 0.2333333  0.7777778 0.3000000 1.555556     7
9                 {bodywash} => {shampoo} 0.2000000  0.6000000 0.3333333 1.500000     6
10                {shampoo} => {bodywash} 0.2000000  0.5000000 0.4000000 1.500000     6
11                   {bodywash} 

In [14]:
# sort rules by a measure such as conf, sup, or lift
sorted_rules <- sort(rules, by="lift", decreasing=TRUE)
inspect(sorted_rules[1:10])

     lhs                               rhs           support   confidence coverage  lift     count
[1]  {bags, paper}                  => {bodywash}    0.2333333 0.7777778  0.3000000 2.333333 7    
[2]  {drinks, electronics, shampoo} => {bags}        0.2000000 1.0000000  0.2000000 2.307692 6    
[3]  {drinks, electronics, paper}   => {bags}        0.2000000 1.0000000  0.2000000 2.307692 6    
[4]  {bags, paper}                  => {cards}       0.2000000 0.6666667  0.3000000 2.222222 6    
[5]  {bags, electronics}            => {drinks}      0.2666667 0.8888889  0.3000000 2.222222 8    
[6]  {bags, shampoo}                => {drinks}      0.2000000 0.8571429  0.2333333 2.142857 6    
[7]  {bags, shampoo}                => {electronics} 0.2333333 1.0000000  0.2333333 2.142857 7    
[8]  {bags, drinks, shampoo}        => {electronics} 0.2000000 1.0000000  0.2000000 2.142857 6    
[9]  {bags, electronics, shampoo}   => {drinks}      0.2000000 0.8571429  0.2333333 2.142857 6    
[10] {bags

In [16]:
# save top 10 rules by lift
sorted_rules_df <- as(sorted_rules[1:10], "data.frame")

In [17]:
# inspect
sorted_rules_df

,rules,support,confidence,coverage,lift,count
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
45,"{bags,paper} => {bodywash}",0.2333333,0.7777778,0.3000000,2.333333,7
71,"{drinks,electronics,shampoo} => {bags}",0.2000000,1.0000000,0.2000000,2.307692,6
76,"{drinks,electronics,paper} => {bags}",0.2000000,1.0000000,0.2000000,2.307692,6
42,"{bags,paper} => {cards}",0.2000000,0.6666667,0.3000000,2.222222,6
60,"{bags,electronics} => {drinks}",0.2666667,0.8888889,0.3000000,2.222222,8
50,"{bags,shampoo} => {drinks}",0.2000000,0.8571429,0.2333333,2.142857,6
55,"{bags,shampoo} => {electronics}",0.2333333,1.0000000,0.2333333,2.142857,7
70,"{bags,drinks,shampoo} => {electronics}",0.2000000,1.0000000,0.2000000,2.142857,6
72,"{bags,electronics,shampoo} => {drinks}",0.2000000,0.8571429,0.2333333,2.142857,6


In [20]:
# export
write.csv(rules_df, file="advertise_products.csv", row.names=FALSE)